# Mstrio-py for Administration

In [ ]:
# Importing packages
import pandas as pd
from mstrio.project_objects import list_olap_cubes, OlapCube
from mstrio.object_management import list_folders, Folder
from mstrio.users_and_groups import list_user_groups, list_users, User, UserGroup
from mstrio.modeling.security_filter import list_security_filters, SecurityFilter
from mstrio.access_and_security import security_role
from mstrio.server import Environment, Cluster
from mstrio.connection import Connection
from mstrio.modeling.schema imoprt(
    Attribute,
    AttributeDisplays,
    AttributeForm,
    AttributeSort,
    AttributeSorts,
    DataType,
    FormReference,
    list_attributes,
    ObjectSubType,
    Relationship,
    SchemaManagement,
    SchemaObjectReference,
    SchemaUpdateType
    )
    from mstrio.access_and_security import Privilege

**Connecting to test environment**

In [ ]:
# Credentials fron environment
    env = ****
    user = '****'
    psw = '******'
    
    url = 'https://env-' + str(env) + '.customer.cloud.microstrategy.com/MicroStrategyLibrary/'
    
# Connection to environment
    conn = Connection(url, user, psw, login_mode=1, project_name = 'MicroStrategy Tutorial')

**Creating Users**

In [ ]:
# Individually creating users
    User.create(connection = conn, username = 'pythondemo1', full_name = 'test user1', password = 'fake1234')
    User.create(connection = conn, username = 'pythondemo2', full_name = 'test user1', password = 'fake1234')
    
# Instantiating user objects
    user1_ = User(connection = conn, name = 'test user1')
    user2_ = User(connection = conn, name = 'test user2')

**Creating Users from List**

In [ ]:
# Looping through a list to create multiple users
    users_to_create = [{'username': 'jnawson', 'fullName': 'John Nawson', 'password': 'test1234', 'addresses': 'test@gmail.com'},
                   {'username': 'lmarian', 'fullName': 'Lara Marian', 'password': 'test1234', 'addresses': 'test2@gmail.com'}]
    
    for u in users_to_create:
        User.create(connection = conn, username = u['username'], full_name = u['fullName'], password = u['password'].add_address(name='gmail', address=u['addresses'])

**Creating a User Group**

In [ ]:
# Creating a user group called 'test'
    UserGroup.create(connection=conn, name='Test_Py')
    test_group = Usergroup(connection=conn, name = "Test_Py")
    
    # Adding users to user group
    test_group.add_users(users = [user1_.id, user2_.id])

**Listing Users in a Group and their Privileges**

In [ ]:
# Listing users in a group
    web_users = UserGroup(connection=conn, name='Web Users')
    web_df = pd.DataFrame(web_users.list_members())
    print(web_df)

In [ ]:
    web_users.list_privileges(to_dataFrame=True)

**Privileges**

In [ ]:
# Listing all privileges available
    privilege_dict = Privilege.list_privileges(connection=conn, to_dataFrame=True)
    print(privilege_dict)

In [ ]:
# Creating list of privileges that have 'architect' in them
    architect_privs = []
    
    for priv in privilege_dict:
        if 'Architect' in str(priv['categories']):
            architect_privs.append(priv['id'])
            
    print(architect_privs)

In [ ]:
# Granting architect privileges & security filter to group
 test_group.grant_privilege(architect_privs)

**Alternative - Security Role**

In [ ]:
security_role.list_security_roles(connection=conn)

architect_role = security_role.SecurityRole.create(connection=conn, name = 'Architect Test', privileges=architect_privs)

test_group.assign_security_role(architect_role, project = 'MicroStrategy Tutorial')

**Security Filters**

In [ ]:
# Instantiating a security filter object
    south_sf = SecurityFilter(connection=conn, name='Region is south')
    
# Applying security filter to user group
    test_group.apply_security_filter(south_sf)

**All that code in one line**

In [ ]:
UserGroup.create(connection=conn, name='Test2', members=[user1_, user2_], privilege=privilege_dict). apply_security_filter(south_sf)

**Mass Modify ACLs***

In [ ]:
from mstrio.utils import acl
from mstrio.object_management import folder, object

# Create folder object
test_folder = folder.Folder(connection=conn, id='***************')
#test_folder.acl_add(rights = acl.Rights.BROWSE, trustees=test_group)


# Check folder contents for specific name/prefix
tf_contents = test_folder.get_contents()
for i in tf_contents:
    if 'Test' in i.name:
        print(i)
        i.acl_add(rights = acl.Rights.BROWSE, trustees=test_group)

In [ ]:
all_users = list_users(connection=conn)

for i in all_users:
    if i.addresses:
        print('{:<30s} {:<10s}'.format(i.name, i.addresses[0]['value']))

**Environment Information**

In [ ]:
# Listing projects in the environment
    clust = Cluster(connection = conn)
    clust_prjects = clust.list_projects()
    
    from proj in clust_projects:
        print(proj)